In [ ]:
%load_ext autoreload
%autoreload 2

# mount drive
from google.colab import drive
drive.mount('/content/drive')

# cd into project directory
%cd /content/drive/My\ Drive/Georgia_Tech/Spring_2021/sbic_stereotypes

In [ ]:
!pip install transformers
!pip install datasets
!pip install rouge-score

import torch
import pandas as pd
import numpy as np

from data_preprocessing import *
from datasets import Dataset
from transformers import AutoModelForCausalLM

In [ ]:
#################### PICK MODEL AND DATASET ####################
from_file = 'data/SBIC.v2.dev.csv'
pred_col = ['HITId', 'post', 'sexYN', 'offensiveYN', 'intentYN', 'whoTarget', \
            'targetMinority','targetStereotype', 'speakerMinorityYN']

test_gpt_5epoch = {
                    'TO ACTUAL': 'data/test/sample_gpt_5epoch_dev_actual.csv',
                    'TO PRED': 'data/test/sample_gpt_5epoch_dev_pred.csv',
                    'TRAINED MODEL': 'model/gpt_5epoch/checkpoint-44734/',
                    'BASE MODEL': 'openai-gpt',
                    'SAMPLE SIZE': 2500
                  }

test_gpt2_5epoch = {
                    'TO ACTUAL': 'data/test/sample_gpt2_5epoch_dev_actual.csv',
                    'TO PRED': 'data/test/sample_gpt2_5epoch_dev_pred.csv',
                    'TRAINED MODEL': 'model/gpt2_5epoch/checkpoint-48150/',
                    'BASE MODEL': 'gpt2',
                    'SAMPLE SIZE': 2500
                  }

active_test = test_gpt_5epoch
################################################################
df = pd.read_csv(from_file)

In [ ]:
"""
from testing_utils import *

clean_post(df)
tokenizer = setup_tokenizer(active_test['BASE MODEL'])
model = AutoModelForCausalLM.from_pretrained(active_test['TRAINED MODEL'], \
                                             pad_token_id=tokenizer.eos_token_id)
model.eval()

def get_samples_from_actual(df, pred_col, active_test):
  actual = df[pred_col].sample(n=active_test['SAMPLE SIZE'])
  categorize_var(actual)
  return actual

actual = get_samples_from_actual(df, pred_col, active_test)
predict_samples(model, tokenizer, actual, pred_col, active_test)
""";

In [ ]:
## F1 and Precision/Recall Scores ##
from data_preprocessing import *
from testing_utils import *

actual = pd.read_csv(active_test['TO ACTUAL'])
pred = pd.read_csv(active_test['TO PRED'])

print("Category: (Precision, Recall, F1)")
print('Offensive: ', f1_score(actual, pred, 'offensiveYN', OFFY, OFFN))
print('Intent: ', f1_score(actual, pred, 'intentYN', INTY, INTN))
print('Lewd: ', f1_score(actual, pred, 'sexYN', LEWDY, LEWDN))
print('Group Targeted: ', f1_score(actual, pred, 'whoTarget', GRPY, GRPN))
print('In Group: ', f1_score(actual, pred, 'speakerMinorityYN', INGY, INGN))

In [ ]:
## BLEU/Rouge-L Scores ##
from testing_utils import *

keep_col = ['HITId', 'post', 'sexYN', 'offensiveYN', 'intentYN', 'whoTarget', \
            'speakerMinorityYN']

sub_df = df[['HITId', 'targetMinority', 'targetStereotype']]
sub_df = aggregate_and_format(sub_df)
actual = actual[keep_col].join(sub_df, on='HITId').reindex(columns=pred_col)

references_tm, hypotheses_tm = get_references_and_hypotheses('targetMinority', actual, pred)
bleu_score_tm = get_bleu_score(references_tm, hypotheses_tm)
rouge_scores_tm = get_rouge_scores(references_tm, hypotheses_tm)

references_ts, hypotheses_ts = get_references_and_hypotheses('targetStereotype', actual, pred)
bleu_score_ts = get_bleu_score(references_ts, hypotheses_ts)
rouge_scores_ts = get_rouge_scores(references_ts, hypotheses_ts)

print("Target Minority Scores: ")
print("Bleu Score: ", bleu_score_tm)
print("Rouge Score (Precision, Recall, F1): ", rouge_scores_tm)

print("Implied Stereotype Scores: ")
print("Bleu Score: ", bleu_score_ts)
print("Rouge Score (Precision, Recall, F1): ", rouge_scores_ts)

In [ ]:
total_posts = actual.shape[0]
off_pos = actual[actual['offensiveYN'] == OFFY].shape[0]
int_pos = actual[actual['intentYN'] == INTY].shape[0]
lewd_pos = actual[actual['sexYN'] == LEWDY].shape[0]
grp_pos = actual[actual['whoTarget'] == GRPY].shape[0]
ing_pos = actual[actual['speakerMinorityYN'] == INGY].shape[0]

print("Offensiveness Skew: ", off_pos / total_posts)
print("Intent Skew: ", int_pos / total_posts)
print("Lewdness Skew: ", lewd_pos / total_posts)
print("Group Targeted Skew: ", grp_pos / total_posts)
print("In Group Skew: ", ing_pos / total_posts)